# Day 03: Gemini 3 + ADK

**使用 Gemini 3 和 ADK 构建具有 Google Search grounding 能力的 AI Agent**

## 课程来源
- [Advent of Agents 2025 - Day 3](https://adventofagents.com/)
- [Build an AI Agent with Gemini 3 (Video)](https://www.youtube.com/watch?v=9EGtawwvINs&list=PLOU2XLYxmsIJCVXV1bLV7qnT5hilN3YJ7&index=4&t=1s)
- [ADK Built-in Tools Documentation](https://google.github.io/adk-docs/tools/built-in-tools/)

## 学习目标

1. 理解 Google Search grounding 的概念和价值
2. 掌握 `google_search` 内置工具的使用方法
3. 学会构建能够访问实时信息的 Agent
4. 了解 ADK 工具系统的限制和解决方案
5. 探索 Gemini 3 的新特性

## 为什么需要学习这个？

传统的 LLM 有一个根本性限制：**知识截止日期**。模型只知道训练时的信息，无法获取最新数据。
Google Search Grounding 解决了这个问题，让你的 Agent 能够：
- 回答关于今天新闻的问题
- 查询实时股价、天气、体育比分
- 提供有来源引用的可靠信息

---
## 1. 什么是 Google Search Grounding？

### 1.1 概念解释

**Grounding（接地/锚定）** 是指让 AI 模型能够访问外部数据源，从而提供更准确、更及时的信息。

你可以把它理解为：给 AI 一个"上网搜索"的能力。

```
┌─────────────────────────────────────────────────────────────────────────┐
│                     没有 Grounding 的情况                                │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   用户: "今天比特币价格是多少？"                                          │
│                                                                          │
│   AI: "抱歉，我的知识截止于 2024 年 1 月，                                │
│        无法提供实时价格信息。"                                           │
│                                                                          │
│   问题：AI 只能说"我不知道"，用户体验差                                   │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────┐
│                      有 Grounding 的情况                                 │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   用户: "今天比特币价格是多少？"                                          │
│                                                                          │
│   AI: [自动触发 Google Search 搜索实时价格]                              │
│       "根据最新数据，比特币当前价格约为 $102,345。                         │
│        来源: CoinGecko (2024-12-14 更新)"                                │
│                                                                          │
│   优势：实时数据 + 来源引用 = 可信赖的回答                                │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

### 1.2 Grounding 的核心优势

| 优势 | 说明 | 实际场景 |
|------|------|----------|
| **实时性** | 获取最新信息 | 新闻、价格、天气、比分 |
| **准确性** | 减少 AI 幻觉 | 用真实搜索结果替代猜测 |
| **可验证** | 提供来源引用 | 用户可以点击链接验证 |
| **广泛性** | 整个互联网的知识 | 任何可搜索的信息 |

### 1.3 工作原理

```
用户提问 → Gemini 判断是否需要搜索 → 调用 Google Search API → 获取结果 → 整合生成回答
    │                                        │                       │
    │                                        │                       │
    ▼                                        ▼                       ▼
 "今天新闻"                           搜索"今天新闻"              返回标题、
                                                                  摘要、链接
```

---
## 2. 环境设置

在开始之前，我们需要配置好运行环境。

In [3]:
# ============================================================
# 环境配置
# ============================================================
# 这段代码做了三件事：
# 1. 导入必要的模块
# 2. 设置 Python 路径（让我们能导入 shared 模块）
# 3. 加载 Google API Key
# ============================================================

import sys
import os
from pathlib import Path

# 添加项目根目录到路径，这样我们可以导入 shared 模块
# Path.cwd() 获取当前目录，.parent 获取父目录（项目根目录）
sys.path.insert(0, str(Path.cwd().parent))

# 从 shared 模块加载 API Key
# 这个函数会从 .env 文件读取 GOOGLE_API_KEY
from shared import get_api_key
os.environ["GOOGLE_API_KEY"] = get_api_key()

print("✅ 环境配置完成！")
print(f"   Python 版本: {sys.version.split()[0]}")
print(f"   工作目录: {Path.cwd()}")

✅ 环境配置完成！
   Python 版本: 3.14.2
   工作目录: /Users/tsingwong/Documents/coding/25-Day-Agents-Course-by-Google/day_03


In [4]:
# ============================================================
# 验证 ADK 安装
# ============================================================
# google-adk 是 Google 的 Agent Development Kit
# 它提供了构建 AI Agent 的工具和框架
# ============================================================

import google.adk as adk

print(f"✅ google-adk 已安装")
print(f"   版本: {adk.__version__ if hasattr(adk, '__version__') else '未知'}")

# 💡 学习笔记：
# ADK 版本很重要！google_search 工具在 v0.1.0 后才支持
# 如果遇到问题，先检查版本：pip show google-adk

✅ google-adk 已安装
   版本: 1.21.0


---
## 3. Google Search 工具详解

### 3.1 查看我们的 YAML 配置

先看一下最简单的 Search Agent 长什么样：

In [5]:
# ============================================================
# 查看 YAML Agent 配置
# ============================================================
# YAML 是一种配置文件格式，比 JSON 更易读
# ADK 支持用 YAML 定义 Agent，无需写 Python 代码
# ============================================================

yaml_content = Path("root_agent.yaml").read_text()
print("📄 root_agent.yaml 内容：")
print("=" * 60)
print(yaml_content)
print("=" * 60)

# 💡 学习笔记：
# - name: Agent 的唯一标识符
# - model: 使用的 Gemini 模型
# - instruction: 系统提示词，定义 Agent 的行为
# - tools: Agent 可以使用的工具列表
#
# 注意 tools 下面只有一个 google_search
# 这是 ADK 的内置工具，不需要额外配置

📄 root_agent.yaml 内容：
name: search_agent
model: gemini-2.5-flash
description: A helpful assistant that can search the web for current information.
instruction: |
  You are a helpful assistant with access to Google Search.

  Use Google Search when users ask about:
  - Current events and news
  - Weather information
  - Stock prices and market data
  - Sports scores and results
  - Any time-sensitive information

  Always provide clear, accurate responses based on search results.
  Cite your sources when appropriate.
tools:
  - name: google_search



### 3.2 Google Search 工具规格

| 属性 | 值 | 说明 |
|------|------|------|
| **工具名称** | `google_search` | 在 YAML 或 Python 中使用这个名称 |
| **类型** | 内置工具 (Built-in Tool) | Google 官方提供，无需自己实现 |
| **兼容模型** | Gemini 2.0+ | 只能与 Gemini 2 及更高版本一起使用 |
| **功能** | 执行 Google 搜索 | 返回搜索结果摘要和链接 |
| **调用方式** | 自动 | Gemini 自动决定何时需要搜索 |

### 3.3 YAML 配置格式（重要！）

在 YAML 中配置工具时，**必须使用 `name` 字段**：

```yaml
# ✅ 正确格式
tools:
  - name: google_search

# 如果工具需要参数，用 args 字段
tools:
  - name: AgentTool
    args:
      agent: ./another_agent.yaml
```

ADK 支持的工具类型：
1. **内置工具**: `google_search`, `code_execution` 等
2. **自定义函数**: `my_module.my_function`
3. **自定义类**: `my_module.MyToolClass`

### 3.4 重要限制（必须知道！）

⚠️ **这些限制很重要，会影响你的 Agent 设计：**

| 限制 | 说明 | 解决方案 |
|------|------|----------|
| **只支持 Gemini** | 不能用于 OpenAI、Claude 等模型 | 如需其他模型，自己实现搜索工具 |
| **不能混用** | 内置工具不能与自定义 Python 工具混用 | 使用多 Agent 架构（后面会讲） |
| **每 Agent 一个** | 一个 Agent 只能用一个内置工具 | 多个功能 = 多个子 Agent |

```
❌ 错误示例：
tools:
  - name: google_search    # 内置工具
  - name: my_custom_tool   # 自定义工具（不能混用！）
  
✅ 正确做法：使用子 Agent（见第 6 节）
```

---
## 4. 用 Python 创建 Search Agent

YAML 适合快速原型，但 Python 给我们更多控制权。让我们用 Python 创建同样的 Agent：

In [ ]:
# ============================================================
# 用 Python 创建 Search Agent
# ============================================================
# 这段代码等同于 root_agent.yaml 的配置
# 但用 Python 写可以：
# - 动态修改配置
# - 添加条件逻辑
# - 更好的 IDE 支持和调试
# ============================================================

from google.adk.agents import Agent
from google.adk.tools import google_search  # 导入内置的搜索工具

# 创建带 Google Search 能力的 Agent
search_agent = Agent(
    # name: Agent 的唯一标识符，用于日志和调试
    name="search_agent",

    # model: 指定使用的 Gemini 模型
    # gemini-2.0-flash 是快速且经济的选择
    # 也可以用 gemini-2.0-pro 获得更好质量
    model="gemini-2.0-flash",

    # instruction: 系统提示词，定义 Agent 的行为
    # 这是 Agent 的"性格"和"工作指南"
    instruction="""You are a helpful assistant with access to Google Search.

Use Google Search when users ask about:
- Current events and news (今日新闻)
- Weather information (天气预报)
- Stock prices and market data (股票价格)
- Sports scores and results (体育比分)
- Any time-sensitive information (任何时效性信息)

Always provide clear responses and cite sources when appropriate.
Respond in the user's language (用用户的语言回复).""",

    # tools: Agent 可以使用的工具
    # google_search 是从 google.adk.tools 导入的内置工具
    tools=[google_search]
)

# 打印 Agent 信息确认创建成功
print(f"✅ Search Agent 创建成功！")
print(f"   名称: {search_agent.name}")
print(f"   模型: {search_agent.model}")
print(f"   工具数量: {len(search_agent.tools)}")

# 💡 学习笔记：
# Agent 类是 ADK 的核心，它封装了：
# - 与 Gemini API 的通信
# - 工具调用的处理
# - 对话历史的管理

✅ Search Agent 创建成功！
   名称: search_agent
   模型: gemini-2.0-flash
   工具数量: 1


### 4.1 运行 Search Agent

创建了 Agent 后，我们需要用 Runner 来运行它。ADK 提供了几种 Runner：
- `InMemoryRunner`: 内存中运行，适合测试
- `VertexAIRunner`: 连接 Vertex AI，适合生产

In [ ]:
# ============================================================
# 运行 Search Agent
# ============================================================
# InMemoryRunner 是最简单的运行方式
# 它在内存中管理会话，适合开发和测试
# ============================================================

from google.adk.runners import InMemoryRunner
from google.genai.types import Content, Part

async def run_search_agent(query: str):
    """
    运行 Search Agent 并返回结果

    Args:
        query: 用户的问题
    """
    # 创建 Runner
    # app_name 用于标识应用，可以是任意字符串
    runner = InMemoryRunner(
        agent=search_agent,
        app_name="day_03_demo"
    )

    # 创建会话
    # 会话保存对话历史，让 Agent 能记住上下文
    session = await runner.session_service.create_session(
        app_name="day_03_demo",
        user_id="user_1"  # 用户标识符
    )

    # 构造用户消息
    # Content 和 Part 是 Gemini API 的消息格式
    user_message = Content(
        role="user",
        parts=[Part(text=query)]
    )

    print(f"👤 用户: {query}")
    print("-" * 50)

    # 运行 Agent 并获取流式响应
    # run_async 是异步生成器，逐步返回事件
    async for event in runner.run_async(
        user_id="user_1",
        session_id=session.id,
        new_message=user_message
    ):
        # 检查事件是否包含文本内容
        if event.content and event.content.parts:
            for part in event.content.parts:
                if hasattr(part, 'text') and part.text:
                    print(f"🤖 Agent: {part.text}")

    print("\n" + "=" * 50)

# 💡 学习笔记：
# - async/await 是 Python 的异步编程语法
# - 流式响应让我们能看到 Agent "思考"的过程
# - 在 Jupyter 中可以直接 await，普通 Python 需要 asyncio.run()

In [6]:
# ============================================================
# 测试 1: 中文查询 - 科技新闻
# ============================================================
# 这个查询会触发 Google Search
# 因为它询问的是"今天"的新闻，需要实时信息
# ============================================================

await run_search_agent("今天的科技新闻有哪些？")

# 💡 观察：
# - Agent 是否调用了 Google Search？
# - 返回的新闻是否是最新的？
# - 有没有提供来源链接？

👤 用户: 今天的科技新闻有哪些？
--------------------------------------------------
🤖 Agent: 以下是今天的一些科技新闻：

*   **苹果折叠屏手机:** 苹果即将推出的折叠屏手机 "iPhone Ultra" 预计售价接近2400美元。
*   **全球人工智能大会:** 非洲正在突尼斯首都苏塞举办全球人工智能大会。
*   **Windows 11 升级问题:** Windows 11 严格的升级要求导致数百万台电脑无法兼容。
*   **太空军事化:** 美国军方警告说，中俄正在积极测试卫星隐形技术。
*   **无人机技术:** 美军正在测试 LUCAS 自杀无人机。
*   **AI 半导体盈利:** 目前只有英伟达和台积电在 AI 半导体领域真正获利。
*   **中国在关键技术研究领域领先:** 《自然》杂志报道，中国在90%的关键技术研究中全球领先。
*   **人形机器人:** 全国首家人形机器人7S店亮相。
*   **南极考察:** 中国“雪鹰601”飞机在南极完成了10年航程。

请注意，由于时间限制，以上信息可能不包含所有最新的科技新闻。您可以参考我提供的链接获取更多信息。



In [7]:
# ============================================================
# 测试 2: 英文查询 - 天气
# ============================================================
# 天气信息是典型的需要实时数据的场景
# ============================================================

await run_search_agent("What is the current weather in Tokyo?")

# 💡 观察：
# - Agent 能处理英文查询
# - 返回的是当前天气，不是历史数据

👤 用户: What is the current weather in Tokyo?
--------------------------------------------------
🤖 Agent: The current weather in Tokyo, Japan is clear with a temperature of 42°F (6°C), but it feels like 41°F (5°C). The humidity is around 83%.




In [8]:
# ============================================================
# 测试 3: 不需要搜索的查询
# ============================================================
# 有些问题不需要搜索，Gemini 可以直接回答
# Agent 会自动判断是否需要调用工具
# ============================================================

await run_search_agent("什么是机器学习？用一句话解释。")

# 💡 观察：
# - 这是一个知识性问题，不需要实时搜索
# - Gemini 应该直接从训练知识中回答
# - Agent 智能地决定何时使用工具

👤 用户: 什么是机器学习？用一句话解释。
--------------------------------------------------
🤖 Agent: 机器学习是人工智能的一个分支，它使计算机能够从数据中学习并进行决策或预测，而无需进行明确的编程。 机器学习专注于开发通过经验和使用数据自动改进的计算机算法.




---
## 5. 直接使用 GenAI SDK 调用 Search Grounding

除了 ADK，我们也可以直接用更底层的 `google-genai` SDK。这种方式让我们能看到更多细节。

In [9]:
# ============================================================
# 使用 GenAI SDK 直接调用 Search Grounding
# ============================================================
# 这是更底层的方式，不通过 ADK Agent
# 优点：可以看到原始的 API 响应和元数据
# 缺点：需要自己管理对话历史和工具调用
# ============================================================

from google import genai
from google.genai import types

# 创建客户端
client = genai.Client()

# 配置 Google Search 工具
# 这里直接构造 Tool 对象，而不是导入
google_search_tool = types.Tool(
    google_search=types.GoogleSearch()  # 空配置表示使用默认设置
)

# 发送请求
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="What are the latest AI news today?",
    config=types.GenerateContentConfig(
        tools=[google_search_tool]  # 传入工具配置
    )
)

print("🔍 Google Search Grounding 结果：")
print("=" * 60)
print(response.text)

# 💡 学习笔记：
# GenAI SDK 是 Gemini API 的 Python 客户端
# types.Tool 定义工具配置
# types.GoogleSearch() 是搜索工具的具体配置

🔍 Google Search Grounding 结果：
Here's a summary of the latest AI news:

**AI Developments & Applications**

*   **AI-powered Dementia Detection:** Researchers have developed new AI systems that analyze EEG data to distinguish between healthy individuals and those with dementia, achieving high accuracy.
*   **Alibaba's Quark AI Glasses:** Alibaba has launched Quark AI glasses with real-time translation, object recognition, and integration with Alibaba's ecosystem.
*   **NYC Council AI Oversight:** New York City Council has created an office to monitor AI systems used by city agencies, focusing on transparency and responsible AI use.
*   **AI-powered Banking:** BBVA is embedding AI into banking workflows using ChatGPT Enterprise. Malaysia has launched Ryt Bank, its first AI-powered bank.
*   **AI Browser:** OpenAI has launched "Atlas," an AI-powered web browser with an AI assistant for summarizing information and automating online tasks.
*   **AI Video Creation:** Google's Veo 3 AI video 

In [ ]:
# ============================================================
# 查看 Grounding 元数据
# ============================================================
# 响应中包含元数据，告诉我们搜索了什么、引用了哪些来源
# 这在生产环境中很重要：可以向用户展示来源
# ============================================================

print("\n📊 Grounding 元数据分析：")
print("=" * 60)

# 检查是否有 grounding 元数据
if response.candidates and response.candidates[0].grounding_metadata:
    metadata = response.candidates[0].grounding_metadata

    # 搜索入口点（用户看到的搜索链接）
    if hasattr(metadata, 'search_entry_point') and metadata.search_entry_point:
        print(f"🔗 搜索入口点: {metadata.search_entry_point}")

    # 引用的网页片段
    if hasattr(metadata, 'grounding_chunks') and metadata.grounding_chunks:
        print(f"\n📚 引用来源 ({len(metadata.grounding_chunks)} 个):")
        for i, chunk in enumerate(metadata.grounding_chunks[:5]):  # 最多显示 5 个
            if hasattr(chunk, 'web'):
                print(f"   [{i+1}] {chunk.web.title}")
                print(f"       {chunk.web.uri}")

    # 支持得分（可选）
    if hasattr(metadata, 'grounding_supports') and metadata.grounding_supports:
        print(f"\n📈 支持得分: {len(metadata.grounding_supports)} 个引用支持")
else:
    print("❌ 没有 Grounding 元数据（可能没有触发搜索）")

# 💡 学习笔记：
# grounding_metadata 包含：
# - search_entry_point: 搜索链接，可以嵌入到 UI 中
# - grounding_chunks: 引用的网页片段
# - grounding_supports: 每个引用的置信度得分
#
# 在生产环境中，你应该向用户展示这些来源，增加可信度


📊 Grounding 元数据分析：
🔗 搜索入口点: rendered_content='<style>\n.container {\n  align-items: center;\n  border-radius: 8px;\n  display: flex;\n  font-family: Google Sans, Roboto, sans-serif;\n  font-size: 14px;\n  line-height: 20px;\n  padding: 8px 12px;\n}\n.chip {\n  display: inline-block;\n  border: solid 1px;\n  border-radius: 16px;\n  min-width: 14px;\n  padding: 5px 16px;\n  text-align: center;\n  user-select: none;\n  margin: 0 8px;\n  -webkit-tap-highlight-color: transparent;\n}\n.carousel {\n  overflow: auto;\n  scrollbar-width: none;\n  white-space: nowrap;\n  margin-right: -12px;\n}\n.headline {\n  display: flex;\n  margin-right: 4px;\n}\n.gradient-container {\n  position: relative;\n}\n.gradient {\n  position: absolute;\n  transform: translate(3px, -9px);\n  height: 36px;\n  width: 9px;\n}\n@media (prefers-color-scheme: light) {\n  .container {\n    background-color: #fafafa;\n    box-shadow: 0 0 0 1px #0000000f;\n  }\n  .headline-label {\n    color: #1f1f1f;\n  }\n  .chip {\n    b

---
## 6. 多 Agent 架构：解决工具混用限制

### 6.1 问题回顾

前面提到，ADK 有一个限制：**内置工具不能与自定义工具混用**。

但实际场景中，我们往往需要 Agent 同时具备：
- 搜索能力（用 `google_search`）
- 计算能力（自定义的计算函数）
- 数据库查询（自定义的 DB 工具）

### 6.2 解决方案：多 Agent 架构

```
                    ┌─────────────────┐
                    │   协调 Agent    │
                    │  (Coordinator)  │
                    └────────┬────────┘
                             │
           ┌─────────────────┼─────────────────┐
           │                 │                 │
           ▼                 ▼                 ▼
    ┌─────────────┐   ┌─────────────┐   ┌─────────────┐
    │ Search Agent│   │ Calculator  │   │   其他...    │
    │(google_search)│  │  Agent      │   │             │
    └─────────────┘   └─────────────┘   └─────────────┘
```

核心思想：**把不同能力分配给不同的子 Agent，由协调 Agent 统一调度**。

In [ ]:
# ============================================================
# 多 Agent YAML 配置示例
# ============================================================
# 这个配置展示了如何用 YAML 定义多 Agent 系统
# 协调 Agent 管理两个子 Agent：搜索和计算
# ============================================================

multi_agent_yaml = '''# root_agent.yaml - 多 Agent 架构示例

name: coordinator
model: gemini-2.0-flash
description: 协调搜索和计算任务的智能助手
instruction: |
  You are a coordinator agent that manages specialized sub-agents.
  你是一个协调 Agent，管理专门的子 Agent。

  任务分配规则：
  - 当前事件、新闻、实时数据 → 使用 search_agent
  - 数学计算、公式求解 → 使用 calculator_agent

  综合子 Agent 的回答，给用户一个完整的响应。

sub_agents:
  # 子 Agent 1: 搜索专家
  - name: search_agent
    model: gemini-2.0-flash
    description: 专门负责网络搜索，获取实时信息
    instruction: |
      You are a search specialist.
      Search the web for accurate, up-to-date information.
      Always cite your sources.
    tools:
      - google_search  # 内置工具

  # 子 Agent 2: 计算专家
  - name: calculator_agent
    model: gemini-2.0-flash
    description: 专门负责数学计算
    instruction: |
      You are a math specialist.
      Help with calculations and show your work step by step.
    tools:
      - tools.calculate  # 自定义工具（需要 tools.py 文件）
'''

print("📄 多 Agent 配置示例：")
print("=" * 60)
print(multi_agent_yaml)

# 💡 学习笔记：
# sub_agents 是关键！它让一个 Agent 可以"调用"其他 Agent
# 每个子 Agent 可以有自己的工具，互不干扰
# 协调 Agent 根据任务类型，自动选择合适的子 Agent

📄 多 Agent 配置示例：
# root_agent.yaml - 多 Agent 架构示例

name: coordinator
model: gemini-2.0-flash
description: 协调搜索和计算任务的智能助手
instruction: |
  You are a coordinator agent that manages specialized sub-agents.
  你是一个协调 Agent，管理专门的子 Agent。

  任务分配规则：
  - 当前事件、新闻、实时数据 → 使用 search_agent
  - 数学计算、公式求解 → 使用 calculator_agent

  综合子 Agent 的回答，给用户一个完整的响应。

sub_agents:
  # 子 Agent 1: 搜索专家
  - name: search_agent
    model: gemini-2.0-flash
    description: 专门负责网络搜索，获取实时信息
    instruction: |
      You are a search specialist.
      Search the web for accurate, up-to-date information.
      Always cite your sources.
    tools:
      - google_search  # 内置工具

  # 子 Agent 2: 计算专家
  - name: calculator_agent
    model: gemini-2.0-flash
    description: 专门负责数学计算
    instruction: |
      You are a math specialist.
      Help with calculations and show your work step by step.
    tools:
      - tools.calculate  # 自定义工具（需要 tools.py 文件）



In [ ]:
# ============================================================
# 用 Python 实现多 Agent 系统
# ============================================================
# 这里我们用 Python 实现同样的多 Agent 架构
# 比 YAML 更灵活，可以添加复杂的自定义工具
# ============================================================

from google.adk.agents import Agent
from google.adk.tools import google_search

# -------------------- 自定义工具 --------------------

def calculate(expression: str) -> str:
    """
    计算数学表达式。

    这是一个自定义工具，Agent 可以调用它来进行数学计算。

    Args:
        expression: 数学表达式，如 "2 + 2"、"sqrt(16)"、"sin(pi/2)"

    Returns:
        计算结果的字符串形式
    """
    import math
    try:
        # 安全的数学函数白名单
        # 我们只允许这些函数，防止代码注入
        safe_dict = {
            "sqrt": math.sqrt,
            "sin": math.sin,
            "cos": math.cos,
            "tan": math.tan,
            "pi": math.pi,
            "e": math.e,
            "log": math.log,
            "log10": math.log10,
            "pow": math.pow,
            "abs": abs,
            "round": round,
        }
        # eval 在受控环境下执行表达式
        result = eval(expression, {"__builtins__": {}}, safe_dict)
        return f"计算结果: {result}"
    except Exception as e:
        return f"计算错误: {e}"

# -------------------- 子 Agent 1: 搜索专家 --------------------

search_sub_agent = Agent(
    name="search_agent",
    model="gemini-2.0-flash",
    instruction="""You are a search specialist.
Search the web for accurate, up-to-date information.
Always cite your sources with URLs when possible.""",
    tools=[google_search]  # 只有搜索工具
)

# -------------------- 子 Agent 2: 计算专家 --------------------

calculator_sub_agent = Agent(
    name="calculator_agent",
    model="gemini-2.0-flash",
    instruction="""You are a math specialist.
Help users with mathematical calculations.
Show your work step by step.
Use the calculate tool for complex expressions.""",
    tools=[calculate]  # 只有计算工具
)

# -------------------- 协调 Agent --------------------

coordinator = Agent(
    name="coordinator",
    model="gemini-2.0-flash",
    instruction="""You are a coordinator that manages specialized sub-agents.

Task routing rules:
- For current events, news, real-time data → use search_agent
- For math calculations, formulas → use calculator_agent

Synthesize responses from sub-agents into coherent answers.
Respond in the user's language.""",
    sub_agents=[search_sub_agent, calculator_sub_agent]  # 注册子 Agent
)

print("✅ 多 Agent 系统创建成功！")
print(f"   协调者: {coordinator.name}")
print(f"   子 Agent: {[a.name for a in coordinator.sub_agents]}")
print()
print("💡 这个系统可以同时处理搜索和计算任务！")

# 💡 学习笔记：
# sub_agents 参数让 Agent 可以调用其他 Agent
# 每个子 Agent 是独立的，有自己的工具和指令
# 协调 Agent 根据任务类型，决定调用哪个子 Agent

✅ 多 Agent 系统创建成功！
   协调者: coordinator
   子 Agent: ['search_agent', 'calculator_agent']

💡 这个系统可以同时处理搜索和计算任务！


---
## 7. Gemini 3 新特性概览

Gemini 3 (也称为 Gemini 2.0) 引入了多项增强 Agent 能力的新特性：

| 特性 | 说明 | Day 3 相关性 |
|------|------|-------------|
| **原生工具支持** | 内置 Google Search、Code Execution 等 | ✅ 今天学的就是这个 |
| **Computer Use** | 可以控制计算机界面（点击、输入） | 后续课程会学习 |
| **实时流式传输** | 支持实时音视频交互 | Live API 课程 |
| **更长上下文** | 支持更大的上下文窗口 | 更复杂的对话 |
| **多模态增强** | 更好的图像/视频理解能力 | 多模态 Agent |

In [13]:
# ============================================================
# 查看可用的 Gemini 模型
# ============================================================
# 了解有哪些模型可用，对选择合适的模型很重要
# ============================================================

from google import genai

client = genai.Client()

print("📋 可用的 Gemini 模型：")
print("=" * 60)

gemini_models = []
for model in client.models.list():
    if 'gemini' in model.name.lower():
        gemini_models.append(model.name)

# 按名称排序
for model_name in sorted(gemini_models):
    print(f"   • {model_name}")

print()
print("💡 提示：")
print("   - flash 模型：速度快、成本低，适合大多数场景")
print("   - pro 模型：质量更高，适合复杂任务")
print("   - 2.0 版本：支持原生工具（如 google_search）")

📋 可用的 Gemini 模型：
   • models/gemini-2.0-flash
   • models/gemini-2.0-flash-001
   • models/gemini-2.0-flash-exp
   • models/gemini-2.0-flash-exp-image-generation
   • models/gemini-2.0-flash-lite
   • models/gemini-2.0-flash-lite-001
   • models/gemini-2.0-flash-lite-preview
   • models/gemini-2.0-flash-lite-preview-02-05
   • models/gemini-2.5-computer-use-preview-10-2025
   • models/gemini-2.5-flash
   • models/gemini-2.5-flash-image
   • models/gemini-2.5-flash-image-preview
   • models/gemini-2.5-flash-lite
   • models/gemini-2.5-flash-lite-preview-09-2025
   • models/gemini-2.5-flash-native-audio-latest
   • models/gemini-2.5-flash-native-audio-preview-09-2025
   • models/gemini-2.5-flash-native-audio-preview-12-2025
   • models/gemini-2.5-flash-preview-09-2025
   • models/gemini-2.5-flash-preview-tts
   • models/gemini-2.5-pro
   • models/gemini-2.5-pro-preview-tts
   • models/gemini-3-pro-image-preview
   • models/gemini-3-pro-preview
   • models/gemini-embedding-001
   • models

---
## 8. 运行 YAML Agent（Web UI 方式）

除了在 Notebook 中运行，我们也可以用 ADK 的 Web UI 来测试 Agent：

### 步骤

```bash
# 1. 打开终端，进入项目根目录
cd /path/to/25-Day-Agents-Course-by-Google

# 2. 启动 ADK Web 服务
uv run adk web

# 3. 打开浏览器
#    http://127.0.0.1:8000

# 4. 从下拉菜单选择 day_03
# 5. 开始对话！
```

### Web UI 的优势

- 更友好的聊天界面
- 可以看到工具调用过程
- 支持多轮对话
- 无需写代码

In [14]:
# ============================================================
# 显示项目结构
# ============================================================

print("📁 Day 03 目录结构：")
print()
print("day-03/")
print("├── __init__.py              # Python 包标记（ADK 需要）")
print("├── root_agent.yaml          # YAML Agent 配置")
print("│                            #   - 定义了 google_search 工具")
print("├── gemini_search_agent.ipynb   # 本 Notebook（教学笔记）")
print("└── README.md                # 学习文档")
print()
print("💡 提示：")
print("   - 文件夹名要用连字符 (day-03)，YAML 中的 name 用下划线")
print("   - __init__.py 是必需的，否则 ADK 找不到这个 Agent")

📁 Day 03 目录结构：

day-03/
├── __init__.py              # Python 包标记（ADK 需要）
├── root_agent.yaml          # YAML Agent 配置
│                            #   - 定义了 google_search 工具
├── gemini_search_agent.ipynb   # 本 Notebook（教学笔记）
└── README.md                # 学习文档

💡 提示：
   - 文件夹名要用连字符 (day-03)，YAML 中的 name 用下划线
   - __init__.py 是必需的，否则 ADK 找不到这个 Agent


---
## 9. 今日总结与收获

### 9.1 核心知识点

| 概念 | 学到了什么 |
|------|------------|
| **Google Search Grounding** | 让 AI 访问实时网络信息，解决知识截止问题 |
| **`google_search` 工具** | ADK 内置工具，只需一行配置即可使用 |
| **YAML 格式** | 工具必须用 `name` 字段：`- name: google_search` |
| **工具限制** | 内置工具不能与自定义工具混用 |
| **多 Agent 架构** | 使用子 Agent 解决工具混用问题 |
| **Grounding 元数据** | 响应中包含来源引用，增加可信度 |

### 9.2 代码模板

**最简 Search Agent（YAML）：**
```yaml
name: search_agent
model: gemini-2.0-flash
instruction: You are a helpful assistant. Use Google Search for current info.
tools:
  - name: google_search   # 注意：必须用 name 字段！
```

**Python 版本：**
```python
from google.adk.agents import Agent
from google.adk.tools import google_search

agent = Agent(
    name="search_agent",
    model="gemini-2.0-flash",
    instruction="Use Google Search for current info.",
    tools=[google_search]  # Python 中直接导入使用
)
```

### 9.3 常见错误

| 错误信息 | 原因 | 解决方案 |
|----------|------|----------|
| `Field required [type=missing]` | YAML 中工具缺少 name 字段 | 改为 `- name: google_search` |
| `Extra inputs are not permitted` | 使用了错误的格式如 `google_search: {}` | 改为 `- name: google_search` |
| `Invalid app name` | 文件夹名包含非法字符 | 使用连字符或下划线 |
| Agent 不出现在下拉菜单 | 缺少 `__init__.py` | 创建空的 `__init__.py` |

### 9.4 实践建议

1. **何时使用 Search**：问题涉及时效性信息时（新闻、价格、天气等）
2. **注意成本**：每次搜索都有 API 调用成本，不要滥用
3. **展示来源**：在生产环境中，向用户展示搜索来源，增加信任
4. **多 Agent 设计**：复杂应用考虑多 Agent 架构，分工明确

---
## 10. 延伸阅读与资源

### 官方文档
- [ADK Built-in Tools](https://google.github.io/adk-docs/tools/built-in-tools/) - 所有内置工具的文档
- [Grounding with Google Search](https://ai.google.dev/gemini-api/docs/grounding) - Grounding 详细指南
- [ADK Python Repository](https://github.com/google/adk-python) - ADK 源代码

### 视频教程
- [Build an AI Agent with Gemini 3 (Video)](https://www.youtube.com/watch?v=9EGtawwvINs&list=PLOU2XLYxmsIJCVXV1bLV7qnT5hilN3YJ7&index=4&t=1s) - Day 3 官方视频
- [Gemini 3 Agent Demo (GitHub)](https://github.com/GoogleCloudPlatform/devrel-demos/tree/main/ai-ml/agent-labs/gemini-3-pro-agent-demo) - 官方示例代码

### 实践练习
- [Google Codelabs: Empowering with Tools](https://codelabs.developers.google.com/devsite/codelabs/build-agents-with-adk-empowering-with-tools) - 动手实验

### 公告
- [Gemini 3 Announcement](https://blog.google/products/gemini-3/#gemini-3) - Gemini 3 官方发布

---

## 下一步

Day 4 将学习更多 ADK 工具和 Agent 能力。继续加油！